In [4]:
import requests

In [ ]:
#https://boardsgenerator.cbs.gov.il/Handlers/WebParts/YishuvimHandler.ashx?dataMode=Yeshuv&language=English&mode=Filters&subject=Population

In [ ]:
#https://boardsgenerator.cbs.gov.il/Handlers/General/Downloader.ashx?type=excel&token=995c7b65-3ea2-44b7-9467-92a29fb1f390

In [77]:
#What i will do -> request 1 -> get years -> iterate for each years + get number of page (iterate for each page) //// concat all data in one json file
# Years -> POST #https://boardsgenerator.cbs.gov.il/Handlers/WebParts/YishuvimHandler.ashx?dataMode=Yeshuv&language=English&mode=Filters&subject=Population
# Data ->  POST #https://boardsgenerator.cbs.gov.il/Handlers/WebParts/YishuvimHandler.ashx?dataMode=Yeshuv&filters={"Years":2021}&filtersearch=&language=English&mode=GridData&pageNumber=1&search=&subject=Population

In [6]:
def get_years():
    try:
        # Sending POST request
        response = requests.post("https://boardsgenerator.cbs.gov.il/Handlers/WebParts/YishuvimHandler.ashx?dataMode=Yeshuv&language=English&mode=Filters&subject=Population")

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse and return the JSON response
            return response.json()
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    except requests.RequestException as e:
        print(f"Request error: {e}")
        return None

In [7]:
years = get_years()["Years"]

In [8]:
def get_number_page(year):
    response = requests.post('https://boardsgenerator.cbs.gov.il/Handlers/WebParts/YishuvimHandler.ashx?dataMode=Yeshuv&filters={"Years":'+str(year)+"}&filtersearch=&language=English&mode=GridData&pageNumber=1&search=&subject=Population")
    
    if response.status_code == 200:
            return response.json()["TotalPages"]
    else:
            return 0

In [9]:
get_number_page(2021)

62

In [26]:
def get_json_file(year,nbrPage):
    try:
        #for i in range(1,nbrPage+1):
            # Sending POST request
            response = requests.post('https://boardsgenerator.cbs.gov.il/Handlers/WebParts/YishuvimHandler.ashx?dataMode=Yeshuv&filters={"Years":'+str(year)+"}&filtersearch=&language=English&mode=GridData&pageNumber="+str(nbrPage)+"&search=&subject=Population")
            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Parse and return the JSON response
                return response.json()
            else:
                #continue
                return None
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return None

In [41]:
get_json_file(2021,2)

{'Columns': [{'Id': 'Semel', 'Name': 'Locality Code', 'Children': None},
  {'Id': 'Name', 'Name': 'Locality name\r\n', 'Children': None},
  {'Id': '1',
   'Name': 'Total',
   'Children': [{'Id': '1_1', 'Name': 'Total', 'Children': None},
    {'Id': '1_2', 'Name': 'Jewish', 'Children': None},
    {'Id': '1_3', 'Name': 'Jewish And Other', 'Children': None},
    {'Id': '1_4', 'Name': 'Arab', 'Children': None}]},
  {'Id': '2',
   'Name': 'Male',
   'Children': [{'Id': '2_1', 'Name': 'Total', 'Children': None},
    {'Id': '2_2', 'Name': 'Jewish', 'Children': None},
    {'Id': '2_3', 'Name': 'Jewish And Other', 'Children': None},
    {'Id': '2_4', 'Name': 'Arab', 'Children': None}]},
  {'Id': '3',
   'Name': 'Female',
   'Children': [{'Id': '3_1', 'Name': 'Total', 'Children': None},
    {'Id': '3_2', 'Name': 'Jewish', 'Children': None},
    {'Id': '3_3', 'Name': 'Jewish And Other', 'Children': None},
    {'Id': '3_4', 'Name': 'Arab', 'Children': None}]}],
 'Table': [{'Semel': '1311',
   'Nam

In [28]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

In [39]:
#concurrent.futures module : provides a high-level interface for asynchronously executing callables
def get_json_file_parallel(year, nbrPage):
    try:
        with ThreadPoolExecutor(max_workers=10) as executor:
            # Use list comprehension to submit tasks to the ThreadPoolExecutor
            tasks = [executor.submit(get_json_file, year, i) for i in range(1, nbrPage + 1)]

            # Iterate through completed futures
            for task in as_completed(tasks):
                try:
                    data = task.result()
                    if data is not None:
                        print(data)
                except Exception as e:
                    print(f"Error fetching page: {e}")

    except Exception as e:
        print(f"Error: {e}")

In [40]:
get_json_file_parallel(2010,1)

{'Columns': [{'Id': 'Semel', 'Name': 'Locality Code', 'Children': None}, {'Id': 'Name', 'Name': 'Locality name\r\n', 'Children': None}], 'Table': [], 'TotalPages': 61, 'TotalCount': 1205}
